In [32]:
import Quartz
import Quartz.CoreGraphics as CG
import PIL.Image as Image
import numpy as np
from skimage import io
import cv2

In [33]:
def take_new_screenshot():
    windows = Quartz.CGWindowListCopyWindowInfo(Quartz.kCGWindowListOptionOnScreenOnly, Quartz.kCGNullWindowID)
    app_window = None

    for window in windows:
        if window.get("kCGWindowOwnerName", "") == "scrcpy":
            app_window = window
            break

    if app_window is None:
        print("Окно приложения не найдено")
        print('Доступные OwnerName: ', [window['kCGWindowOwnerName'] for window in windows])
        print('Замените scrcpy в коде выше на нужное приложение')
        raise SystemExit("Exit")

    # Получить координаты и размеры окна приложения
    bounds = app_window["kCGWindowBounds"]
    x, y, width, height = bounds["X"], bounds["Y"], bounds["Width"], bounds["Height"]

    # Создать контекст изображения
    main_screen = CG.CGMainDisplayID()
    image_ref = CG.CGWindowListCreateImage(CG.CGRectMake(x, y, width, height), CG.kCGWindowListOptionIncludingWindow, app_window["kCGWindowNumber"], CG.kCGWindowImageBoundsIgnoreFraming)

    width = CG.CGImageGetWidth(image_ref)
    height = CG.CGImageGetHeight(image_ref)
    bytesperrow = CG.CGImageGetBytesPerRow(image_ref)

    pixeldata = CG.CGDataProviderCopyData(CG.CGImageGetDataProvider(image_ref))
    image = np.frombuffer(pixeldata, dtype=np.uint8)
    image = image.reshape((height, bytesperrow//4, 4))
    image = image[:,:width,:]

    tmp = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

    im = Image.fromarray(tmp, mode="RGB")
    im.save("screenshot.png")

In [34]:
def parse_figures():
    filename = 'screenshot.png'
    image = io.imread(filename)

    cv2.imshow("Image", cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
    cv2.waitKey(0)


    tmp = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)


    red1 = cv2.inRange(tmp, (0,50,20), (5,255,255))
    red2 = cv2.inRange(tmp, (175,50,20), (180,255,255))
    blue_mask = cv2.inRange(tmp, (110, 100, 100), (130, 255, 255))
    green_mask = cv2.inRange(tmp, (36,25,25), (86, 255,255))

    combined_mask = cv2.bitwise_or(red1, red2)
    combined_mask = cv2.bitwise_or(combined_mask, cv2.bitwise_or(blue_mask, green_mask))

    tmp = cv2.bitwise_and(tmp, tmp, mask=combined_mask)

    cv2.imshow("Image", tmp)
    cv2.waitKey(0)
    cv2.imshow("Image", image)
    cv2.waitKey(0)

In [35]:
# while True:
for i in range(1):
    take_new_screenshot()
    parse_figures()
    cv2.destroyAllWindows()

KeyboardInterrupt: 